In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import random
import pickle
from sklearn.model_selection import KFold
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from sklearn.metrics import accuracy_score, log_loss
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
import matplotlib.pyplot as plt
from tensorflow.keras.applications import MobileNet
from sklearn.metrics import classification_report
from sklearn.utils import class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import LeakyReLU, ReLU
import json

np.random.seed(2020)
random_state = 52
CATEGORIES = ["c0","c1","c2","c3","c4","c5","c6","c7","c8","c9"]

with open('best_params_mobilenet.json') as json_file:
    best_params = json.load(json_file)

In [2]:
def restore_data(path):
    data = dict()
    if os.path.isfile(path):
        file = open(path, 'rb')
        data = pickle.load(file)
    return data

def preprocessing(X, Y, img_rows, img_cols, color_type=3):
    
    Y = np.array(Y, dtype=np.uint8)

    if color_type == 1:
        X = np.array(X, dtype=np.uint8).reshape(-1, img_rows, img_cols, 1)
    else:
        X = np.array(X, dtype=np.uint8).reshape(-1, img_rows, img_cols, 3)

    Y = to_categorical(Y, 10)
    X = X.astype('float32')
    X /= 255
    
    return X, Y


def read_image(path, img_rows, img_cols, color_type=3):
  
  image = cv2.imread(path, cv2.IMREAD_COLOR)
  
  resized_image = cv2.resize(image, (img_cols, img_rows))
  
  return resized_image

def load_images(img_rows, img_cols, color_type, folder):
    
    images = []
    labels = []
    img_names = []

    for category in CATEGORIES:
        path = os.path.join(folder, category)
        label = CATEGORIES.index(category)

        for img in sorted(os.listdir(path)):
            #print(img)
            try:
                image = read_image(os.path.join(path, img), img_rows, img_cols, color_type)     
                images.append(image)
                labels.append(label)
                img_names.append(img)

            except Exception as e:
                pass

    return images, labels, img_names

In [3]:
def best_fine_tuning_mobilenet(img_rows, img_cols, color_type, mobilenet_base):

    model = Sequential()
    model.add(mobilenet_base)
    model.add(Flatten())
    model.add(Dense(best_params["n_units_1"]))
    model.add(ReLU())
    model.add(Dropout(best_params["dropout_rate_1"]))
    model.add(Dense(best_params["n_units_2"]))
    model.add(ReLU())
    model.add(BatchNormalization())
    model.add(Dropout(best_params["dropout_rate_2"]))
    model.add(Dense(10, activation = 'softmax'))

    model.compile(optimizer = SGD(lr=best_params["lr"], momentum=best_params["momentum"]), loss='categorical_crossentropy', metrics = ["accuracy"])

    return model

In [4]:
def training_model(img_rows, img_cols, color_type=3):
    
    mobilenet = MobileNet(weights="imagenet", include_top=False, input_shape=(img_rows, img_cols, color_type))
    model = best_fine_tuning_mobilenet(img_rows, img_cols, color_type, mobilenet)
    
    X_test_1, Y_test_1, img_names_1 = load_images(img_rows, img_cols,color_type,"pilota_1")
    X_test_2, Y_test_2, img_names_2 = load_images(img_rows, img_cols,color_type,"pilota_2")
    X_test_1, Y_test_1 = preprocessing(X_test_1, Y_test_1, img_rows, img_cols, color_type)
    X_test_2, Y_test_2 = preprocessing(X_test_2, Y_test_2, img_rows, img_cols, color_type)
    print("Immagini caricate", len(X_test_1))
    p_test_1_final = []
    p_test_2_final = []

    for k in range(1,6):
        weights_path = os.path.join('best_config_mobilenet', 'weights_kfold_' + str(k) + '.h5')

        model.load_weights(weights_path)

        predictions_test_1 = model.predict_proba(X_test_1, verbose=0)
        predictions_test_2 = model.predict_proba(X_test_2, verbose=0)

        Y_test_1_arg = [y.argmax() for y in Y_test_1]  
        Y_test_2_arg = [y.argmax() for y in Y_test_2]

        p_test_1_final.append(predictions_test_1)
        p_test_2_final.append(predictions_test_2)

    p_test_1_final = np.array(p_test_1_final)
    p_mean_1 = np.mean(p_test_1_final, axis = 0)

    p_test_2_final = np.array(p_test_2_final)
    p_mean_2 = np.mean(p_test_2_final, axis = 0)
    
    predictions_test_1_arg = [y.argmax() for y in p_mean_1]
    predictions_test_2_arg = [y.argmax() for y in p_mean_2] 
    
    print('Classification report pilota_1')
    print(classification_report(Y_test_1_arg, predictions_test_1_arg, target_names=CATEGORIES))

    print('Classification report pilota_2')
    print(classification_report(Y_test_2_arg, predictions_test_2_arg, target_names=CATEGORIES))
    
    return predictions_test_1, img_names_1, predictions_test_2, img_names_2

In [5]:
pred_1, img_names_1, pred_2, img_names_2 = training_model(128,128,3)

Immagini caricate 60
Classification report pilota_1
              precision    recall  f1-score   support

          c0       1.00      0.50      0.67         6
          c1       1.00      1.00      1.00         6
          c2       1.00      1.00      1.00         6
          c3       1.00      1.00      1.00         6
          c4       1.00      1.00      1.00         6
          c5       1.00      1.00      1.00         6
          c6       1.00      1.00      1.00         6
          c7       1.00      1.00      1.00         6
          c8       1.00      1.00      1.00         6
          c9       0.67      1.00      0.80         6

    accuracy                           0.95        60
   macro avg       0.97      0.95      0.95        60
weighted avg       0.97      0.95      0.95        60

Classification report pilota_2
              precision    recall  f1-score   support

          c0       1.00      0.50      0.67         6
          c1       1.00      0.83      0.91      

In [6]:
import pandas as pd

df_1 = pd.DataFrame(np.round(pred_1[0]*1000,0)).transpose()

for i in range(1, len(pred_1)):
    df_1 = pd.concat([df_1,pd.DataFrame(np.round(pred_1[i]*1000,0)).transpose()], axis=0)

df_1["name"] = img_names_1
#df_1 = df_1.sort_values(by=['name'])
df_1.to_csv("demo_v2/demo_1.csv")

In [7]:
df_2 = pd.DataFrame(np.round(pred_2[0]*1000,0)).transpose()

for i in range(1,len(pred_2)):
    df_2 = pd.concat([df_2,pd.DataFrame(np.round(pred_2[i]*1000,0)).transpose()], axis=0)

df_2["name"] = img_names_2
#df_2 = df_2.sort_values(by=['name'])
df_2.to_csv("demo_v2/demo_2.csv")